*Instructors:*
*This Notebook uses the WireDelay.pl script of the Cosmic Ray e-Lab analyses to provide material for the following learning goals:*

* *Unit conversions among nanoseconds, seconds, and days*
* *The 'distance = rate x time' formula*
* *Understanding how measuring devices can affect the precision of scientific data*
* *Estimating reasonable expectations for quantitative results*
* *Reading data files into a program for simple data analysis*

# Wire Delay

## Motivation: Understanding signal travel time

In September of 2011, scientists at the OPERA experiment in Italy announced publicly a result that, if true, would upend nearly all of the past century of physics.  Their detector had measured ghostly particles called neutrinos travelling faster than the speed of light, an observation that directly contradicts the special theory of relativity.

Most scientists, including those on the OPERA team itself, were certain that there must have been some mistake.  Indeed there was, but it was difficult to track down.  After 5 months spent reviewing their experiment's setup, GPS measurements, and data analysis techniques with a fine-toothed comb, the team finally identified two small equipment problems: A clock (one of many) was ticking slightly faster than it was supposed to, and the connector on a fiber-optic cable (also one of many) was loose, causing the signal it carried to be slower than expected.  After fixing these errors, their math was corrected: neutrinos travel slower than light after all, just like relativity requires.
    
This infamous episode in particle physics is a good reminder of an important rule: **scientific researchers must understand how their measuring devices affect the data they gather and be able to communicate this clearly when reporting their observations to others.**

### Wire delay in cosmic ray muon detectors

The same type of equipment errors that bedeviled the OPERA neutrino experiment can also affect the cosmic ray muon detectors (CRMD) used by QuarkNet.  Signals are delivered through cables and connectors from the detector panels to the data acquisition board (DAQ), and from the GPS unit to the DAQ.  Both the GPS unit and the DAQ itself contain clocks that, used together, allow the CRMD to measure the arrival of cosmic ray muons with timing as precise as 10 nanoseconds.  If these elements malfunction or if you don't know how to account for their effects, you may end up with nonsense data -- or, even worse, data that *looks* correct but is subtly flawed so that you end up with the wrong answer without knowing it.

The e-Lab cosmic ray studies use the `WireDelay.pl` data transformation script to ensure that the data used during an analysis correctly accounts for the time it takes for signals to travel between the counter panels and the DAQ on the one hand, and between the GPS unit and the DAQ on the other hand.  In this notebook, we'll see how that's accomplished and see how big of an effect that ends up having on the data you use in cosmic ray studies.

### Abbreviations used in this notebook:

* **CRMD**: **C**osmic **R**ay **M**uon **D**etector
* **DAQ**: **D**ata **A**c**Q**uisition board
* **GPS**: **G**lobal **P**ositioning **S**ystem

## Cosmic Ray Muon Detectors: Clocks and Wires

Fig 1 is a diagram of a CRMD in operation.  A cosmic ray muon has just passed through the four stacked panels of the detector, sending signals from each panel to the DAQ through the wires that connect them.

![The CRMD setup](../Files/Images/DetectorWithGPS.png) *Fig 1: A sketch of the CRMD showing signals traveling from the detector panels to the DAQ as a muon flies through, and the signal traveling from the GPS unit to the DAQ for timekeeping*

If we want to measure the exact time that the muon strikes each panel, we must understand what happens to the electrical signals as they travel through the wires that connect the different elements of the CRMD.

### The speed of signal

You might already be familiar with a very important number:

$$c = 3 \times 10^8 \rm{m/s}\,\rm{.}$$

This is, of course, the speed of light.  Not just the speed of light, though, but the speed of light *in a vacuum*, which is a very important distinction in this case.  When light travels through a medium like water or glass, its speed decreases.  The same is true in the CRMD wires: the signals travel through them at the speed of light, but it's the speed of light in the material the cable is made out of. 

As a rule of thumb, it turns out that this speed is about 2/3 of the speed of light in a vacuum, or

$$v_{\rm wire} = (2/3) c\,\rm{.}$$


We'll do a quick, back-of-the envelope calculation to see how this might affect our CRMD data.  Let's say we're connecting the GPS unit to the DAQ with a cable that's $10\,{\rm m}$ long.  Using the formula

$${\rm distance} = {\rm rate}\times{\rm time}\,{\rm ,}$$

we'll substitute
$${\rm distance} = 10{\rm m}\,{\rm ,}$$
$${\rm rate} = (2/3)c\,{\rm ,}$$

and solve for ${\rm time}$.  No need to grab a calculator, we have Python available right here.

**Exercise 1)** In the code block below, enter a value of 10 meters for the `distance` variable and run the cell to see how long a signal will take to propagate through that length of cable (the number is understood to be in meters, so you don't have to enter the units).

In [ ]:
# Define distance and rate as variables:
distance = #replace me with a value#

# Python, like many programming languages, uses a double-asterisk '**' for exponents:
rate = (2/3)*(3*10**8)

# Solve the formula for time:
time = distance/rate

# Print the result in seconds:
print(time)

**Excercise 2)**  By following the units through the calculation, and by using the definition of a nanosecond, show that the above result of `5e-08` is the same as $50\,{\rm ns}$.

That's $50\,{\rm ns}$ for a signal to travel from the GPS unit to the DAQ over a $10\,{\rm m}$ cable.  If we want to have data that's precise to $10\,{\rm ns}$, then we clearly can't just ignore this $50\,{\rm ns}$ delay due to the signal travel time!  The `WireDelay` data transformation does just that -- it corrects the muon event times recorded in the input data by subtracting the extra time introduced by the wires.  

Before we look at `WireDelay` itself, first we'll think about what we expect to happen, so that we can tell more quickly whether `WireDelay` is working as intended.

## Wire Delays in CRMDs: What We Expect

We've seen how to calculate the time delay given by a wire using the 

$$d=v_{\rm wire}t$$

formula.  Now, we'll go through a couple of thought experiments to help us understand whether we should *add* or *subtract* the calculated delay from the times recorded in the input cosmic ray data.

### 1) Wire Delays in the channel cables

Let's say a cosmic ray muon strikes the panel on channel 1 of the CRMD at exactly 12:00:00 noon.  Imagine that the cable connecting that panel to the DAQ is so long that it takes a full second for the signal to travel to the DAQ, so that the DAQ receives the signal at 12:00:01 pm.

The time we want to record is the time of the muon struck the detector, 12:00:00.  Thus, we conclude that we must *subtract* the wire delay of the detector channels from the DAQ's recorded time of 12:00:01 in order to get the correct time.

**Exercise 3)** Using the formula $\,\,d=v_{\rm wire}t\,\,$ with $\,v_{\rm wire} = (2/3)\,c\,$, calculate how long a cable would have to be for a signal to take 1 second to travel through it.

If you prefer to use Python programming to find the answer, use the following cell:

In [ ]:
# code here
### Instructor's copy:
def wireDelay_TimeToDistance(time):
    '''For input time in seconds, returns the distance in meters for signal in a wire'''
    # In m/s:
    v = (2.0/3.0)*3*10**8
    
    distance = v*time
    return distance

print( str(wireDelay_TimeToDistance(1)) + " meters")

Is your answer reasonable for a cable length?  If you think it's unreasonable, do you think that this changes the conclusion that the wire delay time must be *subtracted* from the recorded time in order to correct it?  Explain why.

### 2) Wire Delays in the GPS cable

Again, let's imagine that the GPS unit is connected to the DAQ with a cable so long that it takes a full second for time data to transmit over it.  When the DAQ wants to know what time it is, it asks the GPS unit, and the GPS responds with a message "It's exactly 5:00:00 pm".  This message travels down the wire, and the DAQ receives it at 5:00:01 pm.  By this time, though, the message is out-of-date!  To find what the correct time is at the moment the DAQ receives the GPS's message, we conclude that we must *add* the wire delay to the time reported by the GPS unit in order to know what time it really is when the message arrives

### 3) Total Wire Delay

We've seen that, in order to correct the cosmic ray muon event times to account for the signal delay in the CRMD wires, we *subtract* the delay due to the wires between the detector panels and the DAQ, and we *add* the delay due to the wires between the GPS unit and the DAQ.  Since these two corrections act in opposite directions, it's possible that the total correction to the cosmic ray data might be either negative (subtracted), positive (added), or even zero if the two types of cable are the same length.

In practice, it's more common for the wire connnecting the GPS unit to the DAQ to be longer than the wires connecting the detector panels to the DAQ.  This is because the GPS unit needs to be near a window or even outside, so that it can receive signals from the GPS satellites, which might be far away from where the DAQ and detector panels are.  

**Exercise 3)** Assume the GPS wire is longer than the detector panel wires.  In this case, what is the sign of the total correction to our cosmic ray time data?  That is, will we end up *adding* it, or *subtracting* it from the raw data?

In [1]:
# Textarea widget
import ipywidgets as widgets
widgets.Text(
    value='',
    description='Answer:',
    disabled=False,
    layout=widgets.Layout(width='100%')
    #layout=widgets.Layout(width='100%', height='200px')
)

Text(value='', description='Answer:', layout=Layout(width='100%'))

**Exercise 4)** Assume one of the wires is $10\,{\rm m}$ longer than the other.  Ignoring the sign, what is the total magnitude of the time correction due to the delay in both types of wire?  *(Hint: you've already done this calculation!)*

In [2]:
# Textarea widget
import ipywidgets as widgets
widgets.Text(
    value='',
    description='Answer:',
    disabled=False,
    layout=widgets.Layout(width='100%')
    #layout=widgets.Layout(width='100%', height='200px')
)

Text(value='', description='Answer:', layout=Layout(width='100%'))

## Using WireDelay.pl

Now we'll look at the `WireDelay` data transformation script that performs the type of correction we've been discussing.  We can execute the Perl script `WireDelay.pl` by calling the Perl interpreter and providing it with the name of the script (which is stored in the `perl/` folder) as well as all the values that the script will need to run:

`$ perl WireDelay.pl <threshIn> <outputs> <geoDir> <daqId> <fw>`&nbsp;&nbsp;.

The items in angled brackets `<>` are parameters we have to specify.  These are:

* `threshIn`:  The name of the Threshold data file we'll use as input.
* `outputs`: What we want to name the script's output file.
* `geoDir`: The directory that contains the detectors' *geometry files*.  These files contain information on the physical layout of the detector panels, including data on wire length.
* `daqId`: The 4-digit ID of the CRMD DAQ that recorded this data.
* `fw`: The version number of the firmware running on the DAQ.  This is included to accommodate older-model DAQs whose firmware worked a bit differently from the current version.  You should always use `1.12` for this value unless you have a specific reason not to. 

For example, we might want to use the Threshold file `6119.2016.0104.1.thresh` as input, which is located in the `../Files/Data/` folder.  The first four digits of the filename tell us that this data was taken by DAQ 6119, so that's our `daqId` value.  We'll name the output file `6119.2016.0104.1.wd` (`wd` for *wire delay*) and write it into the `OutputFiles/` folder.  The geometry file for DAQ 6119 is in the `../Files/Geometry/6119/` folder, but the script already knows to look for a `6119/` folder given the `daqId`, so all we have to give it is the `../Files/Geometry` part.  Putting all of this together,

* `threshIn = ../Files/Data/6119.2016.0104.1.thresh`
* `outputs = OutputFiles/6119.2016.0104.1.wd`
* `geoDir = ../Files/Geometry`
* `daqId = 6119`
* `fw = 1.12`

in which case the command is

`$ perl WireDelay.pl ../Files/Data/6119.2016.0104.1.thresh OutputFiles/6119.2016.0104.1.wd ../Files/Geometry 6119 1.12`

## Understanding WireDelay.pl

To understand what `WireDelay` does, we'll look at a sample of input data, run the script to apply the data transformation, and then look at a sample of output data.  By comparing the input and output data, we'll be able to examine how cosmic ray data is corrected for time delays in the signal wires.

### 1) Investigate the input data

The input file we'll use for this example is `6119.2016.0104.1.thresh`.  We can use the UNIX shell command `head` to get a quick look at what it looks like before the `WireDelay` data transformation acts on it:

In [ ]:
!head -10 ../Files/Data/6119.2016.0104.1.thresh

The first two comment lines (ones that start with `#`) are for technical record-keeping and don't matter to us right now.

The third comment line tells us what each column of data is.  The first column is the DAQ ID and the channel number separated by a period (*channel number* refers to which of the four detector panels produced this row of data).  The second column is the day the data was collected, which we also aren't concerned with in this example.  The main data we'll look at are the third and fourth columns, which record the times when the signal pulse rises above and then falls back below the detector's threshold voltage: 

```
RISING EDGE	       FALLING EDGE
0.3721863017828993	0.3721863017831598
0.3721863017829138	0.3721863017831598
0.3721885846820747	0.3721885846822772
0.3721885846820747	0.3721885846822917
0.3721901866161603	0.3721901866163773
0.3721901866161748	0.3721901866164496
0.3721903650327546	0.3721903650329427
```

_(for more information on the cosmic ray data format and what these numbers mean, there will be another Jupyter Notebook for this)_ 

### 2) Apply the data transformation

Now we'll run `WireDelay` and see what changes it makes to the data.  The script `WireDelay.pl` is a Perl script designed to be executed from a UNIX shell, not from a Jupyter Notebook.  We can use the "!" trick that Jupyter gives us again, though, to call the program using the command that we constructed above:

In [ ]:
!perl ../Files/eLabScripts/WireDelay.pl ../Files/Data/6119.2016.0104.1.thresh OutputFiles/6119.2016.0104.1.wd ../Files/Geometry 6119 1.12

The script has created the output file `OutputFiles/6119.2016.0104.1.wd`.

### 3) Investigate the output data

We can examine the output the same way we did the input file earlier:

In [ ]:
!head -10 OutputFiles/6119.2016.0104.1.wd

So, what has `WireDelay.pl` script done to our data?  Look for differences between this file, `6119.2016.0104.1.wd`, and the input file `6119.2016.0104.1.thresh` shown above.

#### A closer look
First, we can see that the data appears to be in the same order.  The counters are recorded in the same order `6119.1, 6119.3, 6119.2, ...` and with the same "time over threshold" values `22.50, 21.25, 17.50, ...` in both files.  If you're not convinced, increase the number of lines returned by the `head` command until the pattern is clear (the number of lines is given right after the dash in `head -10`).

Next, we note that `WireDelay` keeps the Julian day value from the Threshold file, but it drops the raw-integer rising and falling edge values.  Evidently we won't be needing these values for further analysis.

Last, we notice the most interesting thing: the rising and falling edge time values have changed!  We'll re-write the first seven data lines of each for clarity.

From the input file `6119.2016.0104.1.thresh`:

```
RISING EDGE	       FALLING EDGE
0.3721863017828993	0.3721863017831598
0.3721863017829138	0.3721863017831598
0.3721885846820747	0.3721885846822772
0.3721885846820747	0.3721885846822917
0.3721901866161603	0.3721901866163773
0.3721901866161748	0.3721901866164496
0.3721903650327546	0.3721903650329427
```

From the output file `6119.2016.0104.1.wd`:

```
RISING EDGE	       FALLING EDGE
0.3721978758576562	0.3721978758579167
0.3721978758576707	0.3721978758579167
0.3722001587568317	0.3722001587570342
0.3722001587568317	0.3722001587570486
0.3722017606909173	0.3722017606911343
0.3722017606909317	0.3722017606912065
0.3722019391075115	0.3722019391076997
```

We can use Python to quickly see the difference between values in the output data and corresponding values in the input data:

In [ ]:
inputTimeData = 0.3721863017828993
outputTimeData = 0.3721978758576562

print(outputTimeData - inputTimeData)

Try experimenting -- copy any value from the output file as `outputTimeData` and the corresponding value (by row and column) from the input file as `inputTimeData`, and see what difference you get.

Even better, we can write a short program to read through the input `.thresh` file and the output `.wd` file and find that difference for us.  The following program does just that.

In [ ]:
# Expand this cell to read program 
# Open both files in read-only ('r') mode
with open('../Files/Data/6119.2016.0104.1.thresh', 'r') as beforeFile:
    with open('OutputFiles/6119.2016.0104.1.wd', 'r') as afterFile:

        # Load in the data from the files
        beforeData = beforeFile.readlines()
        afterData = afterFile.readlines()

        # We'll "slice" the data to eliminate header lines and keep 
        #   only what's sufficient to establish the pattern.
        # There are 3 header lines in .thresh files and 1 in .wd files.
        # We'll keep the first 10 data lines of each.
        beforeData = beforeData[3:14]
        afterData = afterData[1:12]

# With the data loaded into memory, we can now find the differences between columns       
# First, print the output headers
print("RISING EDGE (diff)\tFALLING EDGE (diff)\n")

# Read line-by-line through the data
for i in range(len(beforeData)):

    # Split each line into individual values using split().  Our data columns are
    #   tab-separated, so we'll split each line on the tab character '\t'.
    # Rising Edge values are in column 2 (counting from zero).
    # Falling Edge values are in column 3 (counting from zero).
    RE_before = beforeData[i].split('\t')[2]
    FE_before = beforeData[i].split('\t')[3]
    RE_after = afterData[i].split('\t')[2]
    FE_after = afterData[i].split('\t')[3]

    # Values are pulled from the file as text strings.
    # We have to tell Python to interpret them as decimal numbers 
    #   (called "floats") in order to do math.
    RE_diff = float(RE_after) - float(RE_before)
    FE_diff = float(FE_after) - float(FE_before)

    # Now we convert them back to strings to print into a table.
    # We've inclued a special formatting code "{:18.10e}" to make the values
    #   display to the proper number of digits in scientific notation.
    print("{:18.10e}".format(RE_diff) + "\t" + "{:18.10e}".format(FE_diff))


The pattern is unmistakable now: for each line of data, the output file `6119.2016.0104.1.wd` has time values that are `1.1574074757e-05` days greater than the corresponding value in the input file `6119.2016.0104.1.thresh`.  This time represents the wire delay correction, but what exactly does it mean?  We'll need to do a little more work to make it clear.

## Exercise 1: Converting days to seconds

First, we have to confess to a small error.  Recall that the input `.thresh` file had a header that identfied all of the columns:

```
#ID.CHANNEL, Julian Day, RISING EDGE(sec), FALLING EDGE(sec), TIME OVER THRESHOLD (nanosec), RISING EDGE(INT), FALLING EDGE(INT)
6119.1	2457392	0.3721863017828993	0.3721863017831598	22.50	3215689647404250	3215689647406500
6119.3	2457392	0.3721863017829138	0.3721863017831598	21.25	3215689647404375	3215689647406500
```

This header indicates that the `RISING EDGE` and `FALLING EDGE` values (the ones that are $\approx 0.3721863...$) are given in *seconds*.  This is a mistake of the cosmic ray data format that it's too late to fix: the numbers are actually in *days*.  That is, the rising edge of the first line of data occurred $0.3721863017828993$ days after this particular clock started counting.  A value of $0.5000000000000000$ would represent half a day, or 12 hours (or 720 minutes, etc.).  That means that the difference between two `EDGE` times is also in days, including the $0.000011574074756881547$ value that we determined above to be the wire delay correction.

'Days' might be a good unit of time for recording data, but for investigating smaller differences we'll probably want to use more natural units.  Our first task, then, is to convert days into seconds.

**A)** Write a Python function to convert a time value given in days into nanoseconds.  You may name it whatever you like, but I'll suggest `daysToNanosec()`.

In [ ]:
### Write your function here ###

# Hint: If there are 24 hours in a day, what fraction of a day is one hour?
# If there are X nanoseconds in a day, what fraction of a day is one nanosecond?
# What is X?

### These should all be considered part of the 'Instructor's Copy'
def daysToNanosec(days):
    '''A function to convert days into nanoseconds'''
    
    ### 1 day = (24*60*60) sec = (24*60*60) e9 ns
    nanoseconds = days*24*60*60*10**9
    return nanoseconds
    
def daysToSec(days):
    '''A function to convert days into seconds'''

    ### 1 day = (24*60*60) sec
    seconds = days*24*60*60
    return seconds

def wireDelay_TimeToDistance(time):
    '''For input time in seconds, returns the distance in meters for signal in a wire'''
    # In m/s:
    v = (2.0/3.0)*3*10**8
    
    distance = v*time
    return distance

def wireDelay_DistanceToTime(distance):
    '''For input distance in meters, returns the time in seconds for signal in a wire'''
    # In m/s:
    v = (2.0/3.0)*3*10**8

    time = distance/v
    return time

**B)** Apply this function to the wire delay correction of $0.000011574074756881547$ days to see how many seconds it represents.

In [ ]:
print (daysToSec(0.000011574074756881547))

If we've done everything right, it seems like the wire delay correction is just a bit over a single second.  Does that sound reasonable?

Earlier, we used a "1 second" wire delay to get a quick idea of whether we should add or subtract the wire delay correction from the detector panel wires and the GPS unit wires, and you were asked to find the length of a cable that gives that hypothetical wire delay.  In this case, we're looking at the *difference* between the panel cables and the GPS cable, but the question stands: Is this length a reasonable difference to exist between the two cables?

In [ ]:
# Instructor's copy:
print(str(wireDelay_TimeToDistance(1.0000000589945657)) + " m")

Hopefully you found that a wire with a 1-second delay would be around $200,000\,{\rm km}$ in length, which we should all agree is an unreasonable amount -- the circumference of the Earth is only $40,000\,{\rm km}$!  What did we do wrong in our calculation?

The answer is: Nothing!  We've stumbled onto a quirk of the detector's DAQ that there's no way you could have known in advance.  There's a bug in the firmware on the DAQ that causes it to be slow by exactly one second, so the `WireDelay` script was written to add that second back while it corrects for the real, physical effect of the wire delay.

To get the *actual* effect of the wire delay (and only the wire delay) on our cosmic ray data, we'll subtract that second back off.  Subtracting one second from the wire delay correction of $1.0000000589945657\,{\rm s}$ leaves $0.0000000589945657\,{\rm s}$, which is the same as $58.9945657\,{\rm ns}$.

Earlier, we estimated that under normal use of the CRMD, the GPS cable might be approximately $10\,{\rm m}$ longer than the detector panel cables, which would produce a wire delay correction of approximately $+50\,{\rm ns}$.  We weren't that far off!  